In [4]:
import sys,os
import numpy as np
from sklearn.preprocessing import normalize
import scipy
from scipy import sparse
import pyglet
pyglet.options['shadow_window'] = False
import pyrender
import trimesh
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import igl
import meshplot
np.set_printoptions(threshold=np.inf)

**HEAT METHOD for Geodesic Distance Computation**

Name - Vandita Shukla

Student Number - 171415190

Implementation of the paper "The Heat Method for Distance Computation" by Crane, Weischedel and Wardetzky.Submission for Late Summer Assessment

In [153]:
def cotanLaplaceBeltrami(mesh):
    
    """
    
    Distcretized Laplace-Beltrami operator on triangle mesh
    
    """
    
    
    f = mesh.faces
    nFaces = len(mesh.faces)
    
    v = mesh.vertices
    nVerts = len(mesh.vertices)
    
    dEdges = nFaces*6
    cotan = np.zeros(dEdges)
    
    Rw = np.zeros(dEdges)
    Cl = np.zeros(dEdges)
    
    
    for x in range(3):
        
        y = (x+1)%3
        z = (x+2)%3
        
        e1 = v[f[:,y],:] - v[f[:,x],:]
        e2 = v[f[:,z],:] - v[f[:,x],:]
    
        tempC1 = (e1*e2)
        tempC1 = tempC1.sum(axis=1)
        
        tempC2 = np.cross(e1,e2)**2
        tempC2 = np.sum(tempC2,axis=1)**0.5
        
        cotanVal = 0.5*(tempC1/tempC2)
        
        cotan[x*nFaces*2:x*nFaces*2+nFaces] = cotanVal
        Rw[x*nFaces*2:x*nFaces*2+nFaces] = f[:,y]
        Cl[x*nFaces*2:x*nFaces*2+nFaces] = f[:,z]
        
        cotan[x*nFaces*2+nFaces:x*nFaces*2+2*nFaces] = cotanVal
        Rw[x*nFaces*2+nFaces:x*nFaces*2+2*nFaces] = f[:,z]
        Cl[x*nFaces*2+nFaces:x*nFaces*2+2*nFaces] = f[:,y]
        
    Lc = sparse.csr_matrix((cotan,(Rw,Cl)),shape = (nVerts,nVerts))
    Lc = Lc - sparse.spdiags(Lc*np.ones(nVerts),0,nVerts,nVerts) 
    Lc = Lc.tocsr()
        
    #Calculating Area
    edge1 = v[f[:,1]] - v[f[:,0]]
    edge2 = v[f[:,2]] - v[f[:,0]]
    area = 0.5 * (np.sum((np.cross(edge1,edge2))**2,axis=1)**0.5)
    
    vA = np.zeros(nVerts)
    tx = area/3
    
    for i in range(f.shape[1]):
        count = np.bincount(f[:,i].astype(int), tx)
        vA[:len(count)] += count
        
    M = sparse.spdiags(vA,0,nVerts,nVerts)
    
    return Lc, M


In [154]:
def geodesicDistance(mesh, C, A, m, index):
    """
    
    For the computation of geodesic using heat flow algorithm on the mesh
    
    arguments-
    
    mesh - trimesh object
    
    C - Lc from the report Lc^(-1)A yields the Laplace Beltrami operator
    
    A - Diagonal matrix of vertex areas
    
    m - factor for calculation of time step t
    
    idx - starting index on the mesh 
    
    """
    
    #In the next few steps we are calculating the pre-requirements for heat flow algorithm
    vert = mesh.vertices
    fac = mesh.faces
    
    #edges
    e01 = vert[fac[:,1]] - vert[fac[:,0]]
    e12 = vert[fac[:,2]] - vert[fac[:,1]]
    e20 = vert[fac[:,0]] - vert[fac[:,2]]

    TriangleArea = 0.5 * (np.sum((np.cross(e01,e12))**2,axis=1)**0.5)
    
    #(N*ei) for HEAT STEP 2 i.e. required in the gradient calculation of u
    norm = normalize(np.cross(e20,e12))
    unorme01 = normalize(np.cross(norm, e01))
    unorme12 = normalize(np.cross(norm, e12))
    unorme20 = normalize(np.cross(norm, e20))
    
    #For h in time step we calculate the average edge length in the mesh as suggested by Keenan Crane in presentation of the paper   
    e = mesh.edges
    edgesort = np.unique(e,axis=0)
    edge_norms = np.sum((vert[edgesort[:,1]]-vert[edgesort[:,0]])**2, axis=1)**0.5;
    h = np.mean(edge_norms)
    
    #time step
    t = m*h**2
    #t = 0.1
    #pre-factorisation for calculation of u in HEAT STEP 1 and 3
    factoredAtLaplace = scipy.sparse.linalg.splu((A - t * C).tocsc())
    factoredLaplace = scipy.sparse.linalg.splu((C).tocsc())
    
    #setting the value of u0 as 1 at the heat source vertex and zeros elsewhere
    
    u0 = np.zeros(len(vert))
    u0[index] = 1.0
    
    #HEAT METHOD STEP 1
    
    u = np.ravel(factoredAtLaplace.solve(u0),'C')
    #HEAT METHOD STEP 2
    
    grad_u = 1 / (2 * TriangleArea)[:,np.newaxis] * ( unorme01 * u[fac[:,2]][:,np.newaxis]  + unorme12 * u[fac[:,0]][:,np.newaxis] 
            + unorme20 * u[fac[:,1]][:,np.newaxis] )
    
    X = normalize(-grad_u)
    
    #X = -grad_u/(np.sum((grad_u)**2)**0.5)
    
    #HEAT METHOD STEP 3
    
    b = np.zeros(len(vert))
    for x in range(3):
        
        y = (x+1)%3
        z = (x+2)%3
        
        e1 = vert[fac[:,y]] - vert[fac[:,x]]
        e2 = vert[fac[:,z]] - vert[fac[:,x]]
        
        oppositeE = vert[fac[:,y]] - vert[fac[:,z]]
        
        cWeight1 = 1 / np.tan(np.arccos((normalize(-e2) * normalize(oppositeE)).sum(axis=1)))
        
        cWeight2 = 1 / np.tan(np.arccos((normalize(-e1) * normalize(oppositeE)).sum(axis=1)))
         
        #calculation of the vector b containing integrated divergences
        b += np.bincount( (fac[:,x]).astype(int), 0.5 * (cWeight1 * (e1 * X).sum(axis=1) + cWeight2 * (e2 * X).sum(axis=1)), 
                             minlength=len(vert))
        
    phi = np.ravel(factoredLaplace.solve(b))
    phi = phi - np.min(phi)
    
    return phi                 
    

In [155]:
#LOAD BUNNY MESH 
mesh1 = trimesh.load('bunny.obj')

In [156]:
[Lc, M] = cotanLaplaceBeltrami(mesh1)

In [157]:
phi1 = geodesicDistance(mesh1, Lc, M, 1.0, 0)
print

<function print>

In [158]:
#Visualisation
cmap = matplotlib.cm.gist_heat
norm = matplotlib.colors.Normalize(vmin = np.min(phi1), vmax = np.max(phi1))
mesh1.visual.vertex_colors = cmap(norm(phi1))
mesh1.show()

In [159]:
phi2 = geodesicDistance(mesh1, Lc, M, 100.0, 0)

In [160]:
#Visualisation

cmap = matplotlib.cm.gist_heat
norm = matplotlib.colors.Normalize(vmin = np.min(phi2), vmax = np.max(phi2))
mesh1.visual.vertex_colors = cmap(norm(phi2))
mesh1.show()

In [161]:
meshcow = trimesh.load('cow_manifold2.obj')

In [162]:
[Lc3, M3] = cotanLaplaceBeltrami(meshcow)

In [163]:
phi3 = geodesicDistance(meshcow, Lc3, M3, 10.0, 0)

In [164]:
cmap = matplotlib.cm.gist_heat
norm = matplotlib.colors.Normalize(vmin = np.min(phi3), vmax = np.max(phi3))
meshcow.visual.vertex_colors = cmap(norm(phi3))
meshcow.show()